### This is the jupyter notebook for DM&ML Assignment 1 Part B Question 1

#### Q 1.1

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score,auc,accuracy_score,classification_report,confusion_matrix

In [2]:
raw_data = pd.read_excel('Forest.xlsx')
print(raw_data.head())

  class  b1  b2  b3   b4  b5   b6   b7  b8  b9  ...  pred_minus_obs_H_b9  \
0    d   39  36  57   91  59  101   93  27  60  ...                -2.36   
1    h   84  30  57  112  51   98   92  26  62  ...                -2.26   
2    s   53  25  49   99  51   93   84  26  58  ...                -1.46   
3    s   59  26  49  103  47   92   82  25  56  ...                 2.68   
4    d   57  49  66  103  64  106  114  28  59  ...                -2.94   

   pred_minus_obs_S_b1  pred_minus_obs_S_b2  pred_minus_obs_S_b3  \
0               -18.41                -1.88                -6.43   
1               -16.27                -1.95                -6.25   
2               -15.92                -1.79                -4.64   
3               -13.77                -2.53                -6.34   
4               -21.74                -1.64                -4.62   

   pred_minus_obs_S_b4  pred_minus_obs_S_b5  pred_minus_obs_S_b6  \
0               -21.03                -1.60                -6.18  

random_state randomize the samples. for each random state sampling will be different. but for same random state sampling will remain same for each run. 

In [3]:
attributes = raw_data.iloc[:,1:].values
labels = raw_data.iloc[:,0].values
pred_train, pred_test, tar_train, tar_test  = train_test_split(attributes,labels,test_size=0.3,random_state=30,stratify=labels)

In [4]:
classifier_DT = DecisionTreeClassifier(criterion="entropy", random_state=999, min_samples_split=150)
classifier_DT.fit(pred_train,tar_train)
predictions_DT = classifier_DT.predict(pred_test)


In [5]:
print(accuracy_score(tar_test, predictions_DT))
precision = precision_score(y_true=tar_test, y_pred=predictions_DT, average='micro')
print(precision)
recall = recall_score(y_true=tar_test, y_pred=predictions_DT, average='micro')
print(recall)

0.8726114649681529
0.8726114649681529
0.8726114649681529


In [6]:
print(classification_report(tar_test,predictions_DT))

              precision    recall  f1-score   support

          d        0.83      0.94      0.88        48
          h        0.85      0.85      0.85        26
          o        1.00      0.76      0.86        25
          s        0.88      0.88      0.88        58

    accuracy                           0.87       157
   macro avg       0.89      0.86      0.87       157
weighted avg       0.88      0.87      0.87       157



In [7]:
print(confusion_matrix(tar_test,predictions_DT))

[[45  0  0  3]
 [ 0 22  0  4]
 [ 6  0 19  0]
 [ 3  4  0 51]]


In [8]:
MLP_Classifier = MLPClassifier(max_iter=500)
MLP_Classifier.fit(pred_train,np.ravel(tar_train,order='C'))
predictions_MLP = MLP_Classifier.predict(pred_test)
# print(predictions_MLP)

In [9]:
print(classification_report(tar_test,predictions_MLP))

              precision    recall  f1-score   support

          d        0.92      0.94      0.93        48
          h        0.82      0.88      0.85        26
          o        1.00      0.84      0.91        25
          s        0.92      0.93      0.92        58

    accuracy                           0.91       157
   macro avg       0.91      0.90      0.90       157
weighted avg       0.91      0.91      0.91       157



#### Q 1.2

In [10]:
probDT = classifier_DT.predict_proba(pred_test)
probMLP = MLP_Classifier.predict_proba(pred_test)


#### Structure for probabilites array
[
    [
        {d: [DT	MLP]}
        {h: [DT	MLP]}
        {o: [DT	MLP]}
        {s: [DT	MLP]}
    ],
    ...
]


In [11]:
probabilities = []
for i in range(len(pred_test)):
    probabilities.append({'d':[probDT[i][0],probMLP[i][0]],'h':[probDT[i][1],probMLP[i][1]],'o':[probDT[i][2],probMLP[i][2]],'s':[probDT[i][3],probMLP[i][3]]})
firstSampleProbability = probabilities[0]
print("\t DecisionTree \t\t MLP")
for sampleProbability in probabilities:
    for probs in sampleProbability:
        print(probs,'\t',sampleProbability[probs])
    break


	 DecisionTree 		 MLP
d 	 [0.036231884057971016, 0.002944267494963483]
h 	 [0.050724637681159424, 5.6783123588959015e-05]
o 	 [0.007246376811594203, 0.0006896971399021482]
s 	 [0.9057971014492754, 0.9963092522415454]


#### Q 1.3
##### Pseudo code:

#### Q 1.4

In [12]:
def averageProbablePrediction(testSet):
    global np
    testSet = np.atleast_2d(testSet) # if user enters only one sample we convert it to 2D array. 
    averageProbabilities = []
    a,b = classifier_DT.predict_proba(testSet),MLP_Classifier.predict_proba(testSet)
    averages = (np.add(a,b)/2).tolist()
    for avg in averages:
        maxValueIndex = avg.index(max(avg))
        averageProbabilities.append(classifier_DT.classes_[maxValueIndex])
    return np.array(averageProbabilities)
averagePredictions = averageProbablePrediction(pred_test)
print("Accuracy for average aggregate function prediction:",accuracy_score(tar_test,averagePredictions))

Accuracy for average aggregate function prediction: 0.8853503184713376


#### Q 1.5

In [13]:
confusion_matrix(tar_test,predictions_DT)

array([[45,  0,  0,  3],
       [ 0, 22,  0,  4],
       [ 6,  0, 19,  0],
       [ 3,  4,  0, 51]])

In [14]:
"""This function returns conditional probability for each class for a given set of predictions"""
def condProbs(predictions):
    global np
    cm = confusion_matrix(tar_test,predictions)
    colSum = np.sum(cm,axis=0) # gives sum of columns in a 1D array
    correctlyPredicted = cm.diagonal() #gives diagonal elements in 1D array
    return np.divide(correctlyPredicted,colSum) # return element by element division of the two array ultimately giving conditional probability

In [15]:
def conditionalProbabilityPrediction(testSet):
    global np
    conditionalProbabilities = [] # array that will contain all the predicted values at the end 
    dtCondProbs = condProbs(predictions_DT) #gives conditioal probabilty for each class for predictions of DT classifier
    mlpCondProbs = condProbs(predictions_MLP) #gives conditioal probabilty for each class for predictions of MLP classifier
    a,b = classifier_DT.predict_proba(testSet),MLP_Classifier.predict_proba(testSet) 
    for i in range(len(a)):
        dt = a[i].tolist()
        mlp = b[i].tolist()
        p1 = max(a[i]) * dtCondProbs[dt.index(max(a[i]))] # calculate p1 = max of probabilities * conditional probability of getting the class which has max probability for DT
        p2 = max(b[i]) * mlpCondProbs[mlp.index(max(b[i]))] # claculate p2 = same as p2 but with probabilities for MLP
        if p1 > p2:
            conditionalProbabilities.append(MLP_Classifier.classes_[dt.index(max(a[i]))])
        else:
            conditionalProbabilities.append(MLP_Classifier.classes_[mlp.index(max(b[i]))])
    return np.array(conditionalProbabilities)

In [16]:
conditionalPredictions = conditionalProbabilityPrediction(pred_test)
print("Accuracy for conditional prediction:",accuracy_score(tar_test,conditionalPredictions))

Accuracy for conditional prediction: 0.8789808917197452
